# Implement $nCr$ in python
As a variant of Depth First Search

In [4]:
import numpy as np
import pdb
from pprint import pprint


In [5]:
def nprint(*args, header=True):
    if header:
        print("="*80)
    for arg in args:
        print(arg)

## Vertex and Edge classes

In [6]:
class Vertex:
    def __init__(self, vid):
        self.vid = vid # integer
        self.neighbors = set() # set of adjacent Vertex objects
        
    def add_neighbors(self, new_vs):
        for new_v in new_vs:
            self.neighbors.add(new_v)
    def print_neighbors(self):
        for v in self.neighbors:
            print('\t<->', v)
            
    def __eq__(self, other):
        return self.vid == other.vid and self.neighbors == other.neighbors
    
    def __gt__(self, other):
        return self.vid > other.vid
    
    def __ge__(self, other):
        return self == other or self > other
    def __hash__(self):
        return hash(self.vid)
    
    def __str__(self):
        return "Vertex {}".format(self.vid) #": connected to {}".format(self.vid, [n.vid for n in self.neighbors])
    

In [7]:
class UndirectedEdge:
    def __init__(self, v1, v2):
        """
        Args:
        - v1, v2 (Vertex object)
        """
        self.endpoints = set([v1,v2])
        
    def get_endpoints(self, toSort=True, **kwargs):
        epts = list(self.endpoints)
        if toSort:
            epts.sort(**kwargs)
        return epts
    
    def __hash__(self):
        return hash( tuple(self.get_endpoints(toSort=True)) )
        
    def __eq__(self, other):
        if not isinstance(other, UndirectedEdge):
            return NotImplemented
        return self.endpoints == other.endpoints
    
    def __str__(self):
        v1, v2 = self.get_endpoints()
        return "Edge({} <-> {})".format(v1.vid,v2.vid)
        

In [8]:
class DirectedEdge:
    def __init__(self, v1, v2):
        """
        Args:
        - v1, v2 (Vertex object)
        """
        self.endpoints = (v1,v2)
        
    def get_endpoints(self):
        return self.endpoints
    
    def __hash__(self):
        return hash( self.endpoints )
        
    def __eq__(self, other):
        if not isinstance(other, UndirectedEdge):
            return NotImplemented
        return self.endpoints == other.endpoints
    
    def __str__(self):
        v1, v2 = self.endpoints
        return "Edge({} -> {})".format(v1.vid,v2.vid)
        

## Undirected Graph Class

In [9]:
class UndirectedGraph:
    def __init__(self, vertices):
        self.V = {v.vid: v for v in vertices} # set of Vertex objs 
        
        # create self.E based on the adjacency of Vertices in self.V
        self.__init_edges_from_vertices()
        
    def __init_edges_from_vertices(self):
        self.E = set()
        for v in self.V.values():
            for n in v.neighbors:
                assert n.vid in self.V, "Vertex {} not in Graph".format(n.vid)
                self.E.add(UndirectedEdge(v,n))

    def __str__(self):
        pass #todo: tree like printout
    
    def has_vertex(self, v):
        return self.V[v.vid] == v
    
    def has_edge(self, e):
        return e in self.E
    
    def add_vertex(self, v):
        if v.vid in self.V:
            print("No update: Vertex {} already in graph".format(v.vid))
        else: 
            self.V[v.vid] = v
        
    def add_edge(self, e):
        v1, v2 = e.get_endpoints
        if not (v1.vid in self.V and v2.vid in self.V):
            raise ValueError("Both endpoints of the edge must in already in graph. Try add_vertex first")
        
        # Add to graph
        self.E.add(e)
        
        # Add to vertices' `neighbors`
        v1.add_neighbors([v2])
        v2.add_neighbors([v1])
        
        
    def get_neighbors(self, vid):
        return self.V[vid].neighbors
    
    def print_neighbors(self, vid):
        return self.V[vid].print_neighbors()
    
    def print_edges(self):
        for e in self.E:
            print(e)
    def print_structure(self):
        for v in self.V.values():
            print("="*80)
            print(v)
            v.print_neighbors()


## Directed Graph Class

In [10]:
class DirectedGraph:
    """
    Directed graph abstraction that contains a dictionary of Vertex objects
    and a set of DirectedEdge objects
    
    Args:
    - vertices (list): list of Vertex objects
    """
    def __init__(self, vertices):
        self.V = {v.vid: v for v in vertices} # set of Vertex objs 
        
        # create self.E based on the adjacency of Vertices in self.V
        self.__init_edges_from_vertices()
        
    def __init_edges_from_vertices(self):
        self.E = set()
        for v in self.V.values():
            for n in v.neighbors:
                assert n.vid in self.V, "Vertex {} not in Graph".format(n.vid)
                self.E.add(DirectedEdge(v,n))

    def __str__(self):
        pass #todo: tree like printout
    
    def has_vertex(self, v):
        return self.V[v.vid] == v
    
    def has_edge(self, e):
        return e in self.E
    
    def add_vertex(self, v):
        if v.vid in self.V:
            print("No update: Vertex {} already in graph".format(v.vid))
        else: 
            self.V[v.vid] = v
        
    def add_edge(self, e):
        """Adds a directed edge
        """
        if not isinstance(e, DirectedEdge):
            raise TypeError(f"{e} must be of type DirectedEdge: {type(e)}")
        v1, v2 = e.get_endpoints
        if not (v1.vid in self.V and v2.vid in self.V):
            raise ValueError("Both endpoints of the edge must in already in graph. Try add_vertex first")
        
        # Add to graph
        self.E.add(e)
        
        # Add to vertices' `neighbors`
        v1.add_neighbors([v2])        
        
    def get_neighbors(self, vid):
        return self.V[vid].neighbors
    
    def print_neighbors(self, vid):
        return self.V[vid].print_neighbors()
    
    def print_edges(self):
        for e in self.E:
            print(e)
    def print_structure(self):
        for v in self.V.values():
            print("="*80)
            print(v)
            v.print_neighbors()


## DAG Class


In [11]:
class DAG(DirectedGraph):
    """ 
    Fully connected DAG with vertices Vertex(0), Vertex(1), ..., Vertex(n-1)
    
    Args:
    - n (int): number of vertices
    """
    def __init__(self, n):
        verts = [Vertex(i) for i in range(n)]
        # add DAG edges
        for i,v in enumerate(verts):
            v.add_neighbors( [verts[j] for j in range(i+1, n)] )
            
        super().__init__(verts)

## Graph Path Class (ie. Node)

### Node

In [12]:
class Node:
    """
    Node representing a search path on a graph
    
    Args:
    - vid (int): vertex id in a graph 
    - parent (Node): its parent Node object
    
    """
    def __init__(self, vid, parent):
        self.vid = vid
        self.parent = parent 
        
        # depth(ie. level) in the search tree
        # Note a root is at level 1, not zero
        self.path_len = 1 if parent is None else parent.path_len + 1 
        self.depth = self.path_len #alias
        
    def __str__(self):
        clsname =  self.__class__.__name__
        return "{}({}, p={}, depth={})".format(clsname,
                                             self.vid, 
                                            self.parent.vid if self.parent is not None else "None",
                                            self.depth)
# Alternate naming
class Path(Node):
    """
    Args:
    - vid (int): the id of the last vertex of this path
    - parent (Path): Path object that represents the parent path (ie. path leading upto 
        the `end_vid`
    - path_len (int): length of this path (ie. number of vertices on this path(
    """
    def __init__(self, vid, parent):
        super().__init__(vid, parent) #same as above
        
#     def __str__(self):
#         NotImplemented
#         pass #same as above

### CombNode

# TODO: 3/26(W) 9:10PM

# RESUME HERE!

In [13]:
class CombNode(Node):
    """
    Combinatorial node
    Each node instance contains self.vid = a set of elements(not the indices)
    from the original list of elements
    
    Args:
    - vid (frozenset): a set of elements from the orig_list
        - Note this is not a set of indices (integers) indicating the element's index i
    - parent( Node)
    
    Below two are problem-specific 
    - orig_list (list): the original list containing members (not its indices)
    - binsizes (list or tuple): a list of binsizes to divide the elements in orig_list into.
        It must satisfy that sum(binsize) == len(orig_list)
    
    For example, 
    - orig_list = [0,1,2,3,4] 
    - binsize could be (1,1,3) or (1,2,2) if we want to group the elements into three groups
    or (1,4), (2,3) if into 2 groups.
    
    Another example,
    - orig_list = ['highway' , 'primary', 'tertiary', 'residential', 'path', 'cycleway']
    - binsize = (1,1,4), (1,2,3) if we want to group the elements into three groups
    or (1,5), (2,4), (3,3) if into two groups
    """
    def __init__(self, vid, parent, orig_list, binsizes):
        if not isinstance(vid, set):
            raise ValueError("vid must be a set: {}".format(type(vid)))
        if not isinstance(vid, frozenset):
            vid = frozenset(vid)
#         assert np.sum(binsizes) == len(orig_list), "Binsize must sum upto {}".format(len(orig_list))
        

        super().__init__(vid, parent)
        
        if not isinstance(orig_list, np.ndarray):
            orig_list = np.array(orig_list)
        self.orig_list = orig_list
        self.binsizes = binsizes
        
        # element-based (not index-based) list history
        _prev_remaining = self.parent.remaining if self.parent is not None else self.orig_list
        self.remaining = np.array([ele for ele in _prev_remaining if ele not in self.vid])
    
    def get_children(self, verbose=False):
        """
        Returns a list of its children nodes (CombNode objects)
        """
        # If no more remaining elements, return right away
        if len(self.remaining) == 0:
            return []
        
        # Find all possible combinations of size `k` from `self.remaining` list
        n = len(self.remaining)
        k = self.binsizes[self.depth] # children level's binsize since self.depth starts at 1
        idxset_list = nCk(n, k)        
        vidset_list = [ set(self.remaining[ list(idxset) ]) for idxset in idxset_list ]
        
        if verbose:
            print('remaining elements: ', self.remaining)
            print('child binsize: ', k)
            print('idxset_list: ',idxset_list)
            print('vidset_list: ',vidset_list)

        
        # Create CombNodes for children 
        children = [CombNode(vid=vidset, parent=self, orig_list=self.orig_list, binsizes=self.binsizes) 
                    for vidset in vidset_list]
        return children
    
    def __str__(self):
        clsname =  self.__class__.__name__
        p = self.parent.vid if self.parent is not None else "None"
        descr = ( f"{clsname}({self.vid}, p={p}, level={self.path_len})"
#                   f"\n\tOriginal: {self.orig_list}"
                  f", Remaining: {self.remaining}" )
        return descr
        
    

## Traceback function
Print full path from Node $n$ till its root

In [14]:
def get_trace(n, tlist=None):
    """
    Print full path from Node $n$ till its root
    Args:
    - n (Node):
    - tlist (list or None): list of vertex ids (integers)
    """
    if tlist is None: #n is the last node in the trace tree
        tlist = []
    tlist.append(n.vid)
    
    if n.parent is None: # n is the first node in the trace tree. End tracing.
        return tlist
    
    # recurse
    return get_trace(n.parent, tlist)

In [15]:
def test_get_trace():
    v0 = Vertex(0); v1 = Vertex(1); v2 = Vertex(2); v3 = Vertex(3);

    n0 = Node(v0, parent=None)
    n1 = Node(v1, parent=n0)
    n2 = Node(v2, parent=n0)

    n3 = Node(v3, parent=n1)
    print("n0: ", n0)
    print("n1: ", n1)
    print("n2: ", n2)
#     print("trace of n0: ", get_trace(n0))
#     print("trace of n1: ", get_trace(n1))
#     print("trace of n2: ", get_trace(n2))
    print("trace of n3: ", get_trace(n3))

In [16]:
test_get_trace()

n0:  Node(Vertex 0, p=None, depth=1)
n1:  Node(Vertex 1, p=Vertex 0, depth=2)
n2:  Node(Vertex 2, p=Vertex 0, depth=2)
trace of n3:  [<__main__.Vertex object at 0x1101cf6d8>, <__main__.Vertex object at 0x1101ad780>, <__main__.Vertex object at 0x1101adc88>]


## Helper functions to get sample graphs

In [17]:
def get_sample_udgraph():
    v0 = Vertex(0); v1 = Vertex(1); v2 = Vertex(2); v3 = Vertex(3);
    v0.add_neighbors([v1, v2])
    v1.add_neighbors([v0, v2])
    v2.add_neighbors([v1, v3])
    v3.add_neighbors([v2])
    G = UndirectedGraph(vertices=[v0, v1, v2, v3])
    
    return G

def get_sample_udgraph_1():
    v0 = Vertex(0); v1 = Vertex(1); v2 = Vertex(2); v3 = Vertex(3);v4 = Vertex(4);
    v0.add_neighbors([v1, v2, v4])
    v1.add_neighbors([v0, v2])
    v2.add_neighbors([v1, v3])
    v3.add_neighbors([v2])
    v4.add_neighbors([v0])

    G = UndirectedGraph(vertices=[v0, v1, v2, v3, v4])
    
    return G

def get_sample_udgraph_2():
    # Construct grph
    ## todo: better way to add neighbors to both vertices and edges
    v0 = Vertex(0); v1 = Vertex(1); v2 = Vertex(2); v3 = Vertex(3); v4 = Vertex(4); v5 = Vertex(5); v6 = Vertex(6)

    v0.add_neighbors([v1, v4, v6])
    v1.add_neighbors([v0, v2])
    v2.add_neighbors([v1, v3, v4])
    v3.add_neighbors([v2])
    v4.add_neighbors([v0, v2, v5])
    v5.add_neighbors([v4, v6])
    v6.add_neighbors([v0, v5])
    V = [v0, v1, v2, v3, v4, v5, v6]
    
    G = UndirectedGraph(V)
    
    return G
g1 = get_sample_udgraph()
g2 =  get_sample_udgraph_2()

g1.print_structure()

print("+"*80)
g2.print_structure()

Vertex 0
	<-> Vertex 1
	<-> Vertex 2
Vertex 1
	<-> Vertex 0
	<-> Vertex 2
Vertex 2
	<-> Vertex 1
	<-> Vertex 3
Vertex 3
	<-> Vertex 2
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Vertex 0
	<-> Vertex 1
	<-> Vertex 4
	<-> Vertex 6
Vertex 1
	<-> Vertex 0
	<-> Vertex 2
Vertex 2
	<-> Vertex 1
	<-> Vertex 3
	<-> Vertex 4
Vertex 3
	<-> Vertex 2
Vertex 4
	<-> Vertex 0
	<-> Vertex 2
	<-> Vertex 5
Vertex 5
	<-> Vertex 4
	<-> Vertex 6
Vertex 6
	<-> Vertex 0
	<-> Vertex 5


## Tests

### - Vertex, Edge, Graph

In [18]:
def test_Vertex():# Construct some graph
    ## todo: better way to add neighbors to both vertices and edges
    v0 = Vertex(0); v1 = Vertex(1); v2 = Vertex(2); v3 = Vertex(3); v4 = Vertex(4); v5 = Vertex(5); v6 = Vertex(6)

    v0.add_neighbors([v1, v4, v6])
    v1.add_neighbors([v0, v2])
    v2.add_neighbors([v1, v3, v4])
    v3.add_neighbors([v2])
    v4.add_neighbors([v0, v2, v5])
    v5.add_neighbors([v4, v6])
    v6.add_neighbors([v0, v5])
    V = set([v0, v1, v2, v3, v4, v5, v6])

In [19]:
def test_edge():
    e01 = UndirectedEdge(v0,v1); print(e01)
    e10 = UndirectedEdge(v1,v0); print(e10)
    print("e01 == e10: ", e01 == e10)
    e02 = UndirectedEdge(v0,v2); print(e02)

In [20]:
def test_DirectedEdge():
    v0 = Vertex(0); v1 = Vertex(1); v2 = Vertex(2)
    
    e01 = DirectedEdge(v0, v1)
    e02 = DirectedEdge(v0, v2)
    e10 = DirectedEdge(v1, v0)
    print(f"e01: {e01}")
    print(f"e02: {e02}")
    print(f"e01==e02: {e01==e02}")
    print(f"e01==e10: {e01==e10}")

test_DirectedEdge()


e01: Edge(0 -> 1)
e02: Edge(0 -> 2)
e01==e02: False
e01==e10: False


In [22]:
def test_UndirectedGraph():
#     g1 = Graph(vertices=[v0,v1,v2,v3])
    v0 = Vertex(0); v1 = Vertex(1); v2 = Vertex(2); v3 = Vertex(3);
    v0.add_neighbors([v1, v2])
    v1.add_neighbors([v0, v2])
    v2.add_neighbors([v3])
    v3.add_neighbors([v2])
    g1 = UndirectedGraph(vertices=[v0, v1, v2, v3])

    # print([n.vid for n in g1.neighbors(v3)])
    print("Edges:")
    g1.print_edges()
    
    # print adjacent edges
    for vid, v in g1.V.items():
        print("="*80)
        print(v)
        v.print_neighbors()
              
    # check has_vertex and has_edge
    for vid, v in g1.V.items():
        print(v, g1.has_vertex(v))
    dummy = Vertex(0)
    dummy.add_neighbors([v3])
    print("dummy in?: ", g1.has_vertex(dummy))
    
test_UndirectedGraph()


Edges:
Edge(0 <-> 1)
Edge(2 <-> 3)
Edge(0 <-> 2)
Edge(1 <-> 2)
Vertex 0
	<-> Vertex 1
	<-> Vertex 2
Vertex 1
	<-> Vertex 0
	<-> Vertex 2
Vertex 2
	<-> Vertex 3
Vertex 3
	<-> Vertex 2
Vertex 0 True
Vertex 1 True
Vertex 2 True
Vertex 3 True
dummy in?:  False


### - DAG

In [23]:
def test_DAG():
    for k in range(3,7):
        print('\n\nk={}'.format(k))
        dag = DAG(k)
        dag.print_structure()
test_DAG()



k=3
Vertex 0
	<-> Vertex 1
	<-> Vertex 2
Vertex 1
	<-> Vertex 2
Vertex 2


k=4
Vertex 0
	<-> Vertex 1
	<-> Vertex 2
	<-> Vertex 3
Vertex 1
	<-> Vertex 2
	<-> Vertex 3
Vertex 2
	<-> Vertex 3
Vertex 3


k=5
Vertex 0
	<-> Vertex 1
	<-> Vertex 2
	<-> Vertex 3
	<-> Vertex 4
Vertex 1
	<-> Vertex 2
	<-> Vertex 3
	<-> Vertex 4
Vertex 2
	<-> Vertex 3
	<-> Vertex 4
Vertex 3
	<-> Vertex 4
Vertex 4


k=6
Vertex 0
	<-> Vertex 1
	<-> Vertex 2
	<-> Vertex 3
	<-> Vertex 4
	<-> Vertex 5
Vertex 1
	<-> Vertex 2
	<-> Vertex 3
	<-> Vertex 4
	<-> Vertex 5
Vertex 2
	<-> Vertex 3
	<-> Vertex 4
	<-> Vertex 5
Vertex 3
	<-> Vertex 4
	<-> Vertex 5
Vertex 4
	<-> Vertex 5
Vertex 5


### - Node

In [24]:
def test_Node():
    n0 = Node(0, parent=None)
    n1 = Node(1, parent=n0)
    n2 = Node(2, parent=n0)

    n3 = Node(3, parent=n2)
    print("n0: ", n0)
    print("n1: ", n1)
    print("n2: ", n2)
    print("n3: ", n3)
    
def test_Node_2():
    nprint("Path of length 1")
    n0 = Node(0, parent=None);print(n0)
    
    nprint("Paths of length 2")
    n1 = Node(1, parent=n0);print(n1, get_trace(n1))
    n2 = Node(2, parent=n0);print(n2, get_trace(n2))
    n4 = Node(4, parent=n0);print(n4, get_trace(n4))

    nprint("Paths of length 3")
    n3 = Node(3, parent=n2);print(n3, get_trace(n3))
    n5 = Node(4, parent=n2);print(n5, get_trace(n5))
    n6 = Node(5, parent=n4);print(n6, get_trace(n6))

    nprint("Paths of length 4")
    n7 = Node(5, parent=n5);print(n7, get_trace(n7))
           
# test_Node()
test_Node_2()

Path of length 1
Node(0, p=None, depth=1)
Paths of length 2
Node(1, p=0, depth=2) [1, 0]
Node(2, p=0, depth=2) [2, 0]
Node(4, p=0, depth=2) [4, 0]
Paths of length 3
Node(3, p=2, depth=3) [3, 2, 0]
Node(4, p=2, depth=3) [4, 2, 0]
Node(5, p=4, depth=3) [5, 4, 0]
Paths of length 4
Node(5, p=4, depth=4) [5, 4, 2, 0]


Looks great!


---

## Graph Search Algorithms

### 1. Depth First Search

In [25]:
def DFS(G, start_vid, goal_vid,
       verbose=False):
    """
    Args:
    - G (Graph)
        - Has a method `G.neighbors(v)` which returns a list of vertex ids neighboring vertex `v`
    - start_vid (int: Vertex id): vertex id of the start vertex in G
    - goal_vid (int: Vertex id): vertex id of the goal vertex in G
    """
    assert start_vid in G.V, "start node not found in Graph"
    assert goal_vid in G.V, "goal node not found in Graph"
    
    S = Node(vid=start_vid, parent=None)
    Q = [S] # Queue that is actually a stack.  List of Node objs
    Expanded = set() # a set of integers for vertex ids
    while len(Q) > 0:
        N = Q.pop(0)
        if N.vid == goal_vid:
            return get_trace(N)
        
        # Expand N and add its children nodes that haven't been explored yet
        Expanded.add(N.vid)
        cnodes = [Node(vid=c.vid, parent=N) for c in G.get_neighbors(N.vid) if c.vid not in Expanded]
        cnodes.sort(key=lambda n: n.vid)
        Q = cnodes + Q # prepend the children nodes
        
        if verbose:
            print("Expanding: ", N, "...")
            print("Updated stack: ", [node.vid for node in Q])
            print("So far, Expanded: ", Expanded)

#         pdb.set_trace()


    

In [27]:
G = get_sample_udgraph_1()
G.print_structure()

Vertex 0
	<-> Vertex 1
	<-> Vertex 2
	<-> Vertex 4
Vertex 1
	<-> Vertex 0
	<-> Vertex 2
Vertex 2
	<-> Vertex 1
	<-> Vertex 3
Vertex 3
	<-> Vertex 2
Vertex 4
	<-> Vertex 0


In [28]:
DFS(G, 0, 4)

[4, 0]

### 2. kBFS
Find all paths of length $k$ starting from a vertex $V$ in graph $G$
- If $G$ is a DAG with $V = {0,1,...,n-1}$, then this is equivalent to finding all possibile combinations of $n \choose k$

In [29]:
def kBFS(G, start_vid, k, verbose=False):
    """
    Given a graph G=(V,E), find all paths of length `k` starting from vertex `start_vid`
    
    Args:
    - G (Graph)
    - start_vid (int): start vertex to compute the path and path length
    - k (int): length of the paths we are looking for
    
    Returns:
    - collection (list): a list of sets where each set contains `k` integers indicating 
                         vertices on the path of length `k`          
    """
    assert start_vid in G.V, "start node not in the graph"
    
    S = Node(vid = start_vid, parent=None)
    Q = [S] # Queue containing Node objects
#     Expanded = set() # a set of integers for vertex ids visited/expanded 
## 3/26/2019 (w) Keeping track of Expanded node is erronous for BFS
    collection = []
    while len(Q) > 0: 
        N = Q.pop(0)
        
        # check if the search goal is met
        if N.path_len == k:
            collection.append(frozenset(get_trace(N)))

            if verbose:
                print("Woohoo. Found a path of length {}: {}".format(k, N))
                print("\t {}".format(get_trace(N)))
        else:
            # This node needs to be expanded, so that its children paths can be 
            # explored further
            # 3/26/2019 (W): Expanded list is erronous for BFS
#             Expanded.add(N.vid)
            
            # 3/26/2019 (W): Found a bug for `nCn`
            # Below is wrong! nCn will return empty.
#             cnodes = [Node(vid=v.vid, parent=N) for v in G.get_neighbors(N.vid) if v.vid not in Expanded]
            # Instead exclude any node in this node's path trace from the children list
            # Better to implement `get_children` method for Node class
            cnodes = [Node(vid=v.vid, parent=N) for v in G.get_neighbors(N.vid) if v.vid not in get_trace(N)]
        
            # Give order by vertex id
            cnodes.sort(key=lambda n:n.vid)
            Q.extend(cnodes) # add the children nodes to the end of queue (BFS)
            
            if verbose:
                nprint("Expanding: ", N, "...")
                print("Updated queue: ", [node.vid for node in Q])
    
    return collection

### 3. $nCk$ implementation

In [30]:
def nCk(n, k, **kwargs):
    """
    Args:
    - n (int): number of elements in the full list 
    - k (int): number of items to choose
    
    Returns:
    - collections (list): a list of sets where each set contains `k` integers 
        indicating the ids of selected vertices
        
    """
    dag = DAG(n)
#     pdb.set_trace()
    collections = []
    for start_vid in range(n):
        collections.extend(kBFS(dag, start_vid, k, **kwargs))
    return collections
                           
        

### 4. Tests

#### - kBFS

In [31]:
def test_kBFS_1():
    G = get_sample_graph()
    G.print_structure()
    
    result = kBFS(G, 0, 2, verbose=True)
    
def test_kBFS_2():
    G = get_sample_graph()
    G.print_structure()
    
    result = kBFS(G, 0, 3, verbose=True)
    
def test_kBFS_3():
    G = get_sample_graph()
    G.print_structure()
    
    result = kBFS(G, 0, 4, verbose=True)
    
def test_kBFS_4():
    G = get_sample_graph()
    G.print_structure()
    
    result = kBFS(G, 0, 5, verbose=True)  
    print(result)

# test_kBFS_1()
# test_kBFS_2()
# test_kBFS_3()
# test_kBFS_4()


In [32]:
def test_kBFS_DAG_Node_1():
    n = 5 # number of vertices in the graph
    dag = DAG(n)
    dag.print_structure()
    k = 2
    start_vid = 3
    kBFS(dag, start_vid, k, verbose=True)
test_kBFS_DAG_Node_1()

Vertex 0
	<-> Vertex 1
	<-> Vertex 2
	<-> Vertex 3
	<-> Vertex 4
Vertex 1
	<-> Vertex 2
	<-> Vertex 3
	<-> Vertex 4
Vertex 2
	<-> Vertex 3
	<-> Vertex 4
Vertex 3
	<-> Vertex 4
Vertex 4
Expanding: 
Node(3, p=None, depth=1)
...
Updated queue:  [4]
Woohoo. Found a path of length 2: Node(4, p=3, depth=2)
	 [4, 3]


In [33]:
def test_kBFS_DAG_Node_2():
    n = 5 # number of vertices in the graph
    dag = DAG(n); #dag.print_structure()
    
    k = 5
    start_vid = 0
    kBFS(dag, start_vid, k, verbose=True)
test_kBFS_DAG_Node_2()

Expanding: 
Node(0, p=None, depth=1)
...
Updated queue:  [1, 2, 3, 4]
Expanding: 
Node(1, p=0, depth=2)
...
Updated queue:  [2, 3, 4, 2, 3, 4]
Expanding: 
Node(2, p=0, depth=2)
...
Updated queue:  [3, 4, 2, 3, 4, 3, 4]
Expanding: 
Node(3, p=0, depth=2)
...
Updated queue:  [4, 2, 3, 4, 3, 4, 4]
Expanding: 
Node(4, p=0, depth=2)
...
Updated queue:  [2, 3, 4, 3, 4, 4]
Expanding: 
Node(2, p=1, depth=3)
...
Updated queue:  [3, 4, 3, 4, 4, 3, 4]
Expanding: 
Node(3, p=1, depth=3)
...
Updated queue:  [4, 3, 4, 4, 3, 4, 4]
Expanding: 
Node(4, p=1, depth=3)
...
Updated queue:  [3, 4, 4, 3, 4, 4]
Expanding: 
Node(3, p=2, depth=3)
...
Updated queue:  [4, 4, 3, 4, 4, 4]
Expanding: 
Node(4, p=2, depth=3)
...
Updated queue:  [4, 3, 4, 4, 4]
Expanding: 
Node(4, p=3, depth=3)
...
Updated queue:  [3, 4, 4, 4]
Expanding: 
Node(3, p=2, depth=4)
...
Updated queue:  [4, 4, 4, 4]
Expanding: 
Node(4, p=2, depth=4)
...
Updated queue:  [4, 4, 4]
Expanding: 
Node(4, p=3, depth=4)
...
Updated queue:  [4, 4]
Expan

#### - nCk

In [34]:
def test_nCk_1():
    n, k = 5, 2
    collections = nCk(n,k, verbose=True)
    nprint("Done")
    pprint(collections)

def test_nCk_2():
    n, k = 5, 3
    collections = nCk(n,k, verbose=True)
    nprint("Done: n={}, k={}".format(n,k))
    pprint(collections)
    
def test_nCk_3():
    n, k = 5, 5
    collections = nCk(n,k, verbose=True)
    nprint("Done: n={}, k={}".format(n,k))
    pprint(collections)
test_nCk_3()

Expanding: 
Node(0, p=None, depth=1)
...
Updated queue:  [1, 2, 3, 4]
Expanding: 
Node(1, p=0, depth=2)
...
Updated queue:  [2, 3, 4, 2, 3, 4]
Expanding: 
Node(2, p=0, depth=2)
...
Updated queue:  [3, 4, 2, 3, 4, 3, 4]
Expanding: 
Node(3, p=0, depth=2)
...
Updated queue:  [4, 2, 3, 4, 3, 4, 4]
Expanding: 
Node(4, p=0, depth=2)
...
Updated queue:  [2, 3, 4, 3, 4, 4]
Expanding: 
Node(2, p=1, depth=3)
...
Updated queue:  [3, 4, 3, 4, 4, 3, 4]
Expanding: 
Node(3, p=1, depth=3)
...
Updated queue:  [4, 3, 4, 4, 3, 4, 4]
Expanding: 
Node(4, p=1, depth=3)
...
Updated queue:  [3, 4, 4, 3, 4, 4]
Expanding: 
Node(3, p=2, depth=3)
...
Updated queue:  [4, 4, 3, 4, 4, 4]
Expanding: 
Node(4, p=2, depth=3)
...
Updated queue:  [4, 3, 4, 4, 4]
Expanding: 
Node(4, p=3, depth=3)
...
Updated queue:  [3, 4, 4, 4]
Expanding: 
Node(3, p=2, depth=4)
...
Updated queue:  [4, 4, 4, 4]
Expanding: 
Node(4, p=2, depth=4)
...
Updated queue:  [4, 4, 4]
Expanding: 
Node(4, p=3, depth=4)
...
Updated queue:  [4, 4]
Expan

#### - CombNode

In [35]:
def test_CombNode():
    orig_list = [0,1,2,3]
    binsizes = (2,2)
    vidset = set([0,1]) # len is equal to binsizes[0]

    n0 = CombNode(vidset, None, orig_list, binsizes)
    nprint("Root:", n0)

    children = n0.get_children()
    child = children[0]
    nprint("Child1: ", child)
    
    nprint("Check its parent node:")
    print(child.parent)
    
    nprint("Traceback: ",get_trace(child))
test_CombNode()

Root:
CombNode(frozenset({0, 1}), p=None, level=1), Remaining: [2 3]
Child1: 
CombNode(frozenset({2, 3}), p=frozenset({0, 1}), level=2), Remaining: []
Check its parent node:
CombNode(frozenset({0, 1}), p=None, level=1), Remaining: [2 3]
Traceback: 
[frozenset({2, 3}), frozenset({0, 1})]


In [36]:
def test_CombNode_2():
    orig_list = [0,1,2,3,4]
    binsizes = (2,3)
    vidset = set([0,1]) # len is equal to binsizes[0]

    n0 = CombNode(vidset, None, orig_list, binsizes)
    nprint("Root:", n0)
    children = n0.get_children()
    for i, c in enumerate(children):
        nprint(f"Child {i}", c)
    child = children[0]
    nprint("Sample child: ", child)

    nprint("Check its parent node:")
    print(child.parent)

    nprint("Traceback: ",get_trace(child))
test_CombNode_2()

Root:
CombNode(frozenset({0, 1}), p=None, level=1), Remaining: [2 3 4]
Child 0
CombNode(frozenset({2, 3, 4}), p=frozenset({0, 1}), level=2), Remaining: []
Sample child: 
CombNode(frozenset({2, 3, 4}), p=frozenset({0, 1}), level=2), Remaining: []
Check its parent node:
CombNode(frozenset({0, 1}), p=None, level=1), Remaining: [2 3 4]
Traceback: 
[frozenset({2, 3, 4}), frozenset({0, 1})]


In [37]:
def test_CombNode_3():
    """ 
    Check three level of CombNode expansion: Root->child->grandchild
    """
    orig_list = [0,1,2,3,4]
    orig_list = np.array(orig_list)

    binsizes = (2,2,1)
    vidset = set(orig_list[ [0,1]] ) # len is equal to binsizes[0]

    n0 = CombNode(vidset, None, orig_list, binsizes)
    nprint("Root:", n0)

    children = n0.get_children()
    for i, c in enumerate(children):
        nprint(f"Child {i}", c)
        
    child = children[0]
    nprint("Child Sample: ", child)

    nprint("Check its parent node:")
    print(child.parent)
    nprint("Traceback: ",get_trace(child))
    
    grandchildren = child.get_children()
    grandchild = grandchildren[0]
    nprint("Grandchild 1", grandchild)
    nprint("Grandchild Traceback: ",get_trace(grandchild))
    
    # Grandchildren has no child
    ggchild = grandchild.get_children()
    nprint("GGchild: ", ggchild)

test_CombNode_3()

Root:
CombNode(frozenset({0, 1}), p=None, level=1), Remaining: [2 3 4]
Child 0
CombNode(frozenset({2, 3}), p=frozenset({0, 1}), level=2), Remaining: [4]
Child 1
CombNode(frozenset({2, 4}), p=frozenset({0, 1}), level=2), Remaining: [3]
Child 2
CombNode(frozenset({3, 4}), p=frozenset({0, 1}), level=2), Remaining: [2]
Child Sample: 
CombNode(frozenset({2, 3}), p=frozenset({0, 1}), level=2), Remaining: [4]
Check its parent node:
CombNode(frozenset({0, 1}), p=None, level=1), Remaining: [2 3 4]
Traceback: 
[frozenset({2, 3}), frozenset({0, 1})]
Grandchild 1
CombNode(frozenset({4}), p=frozenset({2, 3}), level=3), Remaining: []
Grandchild Traceback: 
[frozenset({4}), frozenset({2, 3}), frozenset({0, 1})]
GGchild: 
[]


In [38]:
def test_CombNode_4():
    """ 
    Check three level of CombNode expansion: Root->child->grandchild
    """
    orig_list = ["zero","one", "two", "three", "four"]
    orig_list = np.array(orig_list)
    binsizes = (2,2,1)
    vidset = set(orig_list[ [0,1]] ) # len is equal to binsizes[0]
    
    n0 = CombNode(vidset, None, orig_list, binsizes)
    nprint("Root:", n0)

    children = n0.get_children()
    for i, c in enumerate(children):
        nprint(f"Child {i}", c)
        
    child = children[0]
    nprint("Child Sample: ", child)

    nprint("Check its parent node:")
    print(child.parent)
    nprint("Traceback: ",get_trace(child))
    
    grandchildren = child.get_children()
    grandchild = grandchildren[0]
    nprint("Grandchild 1", grandchild)
    nprint("Grandchild Traceback: ",get_trace(grandchild))
    
    # Grandchildren has no child
    ggchild = grandchild.get_children()
    nprint("GGchild: ", ggchild)
test_CombNode_4()

def test_CombNode_5():
    orig_list = np.array(["zero","one", "two", "three", "four"])

    # first get the level0 and initiate the queue
    print('orig_list: ', orig_list)
    binsize = [1,1,3]
    print('binsize: ', binsize)

    n = len(orig_list)
    level0_idxset_list = nCk(n,binsize[0])
    level0_vidset_list = [ set(orig_list[list(idxset)])  for idxset in level0_idxset_list ] 
    pprint(level0_vidset_list)

Root:
CombNode(frozenset({'zero', 'one'}), p=None, level=1), Remaining: ['two' 'three' 'four']
Child 0
CombNode(frozenset({'three', 'two'}), p=frozenset({'zero', 'one'}), level=2), Remaining: ['four']
Child 1
CombNode(frozenset({'four', 'two'}), p=frozenset({'zero', 'one'}), level=2), Remaining: ['three']
Child 2
CombNode(frozenset({'four', 'three'}), p=frozenset({'zero', 'one'}), level=2), Remaining: ['two']
Child Sample: 
CombNode(frozenset({'three', 'two'}), p=frozenset({'zero', 'one'}), level=2), Remaining: ['four']
Check its parent node:
CombNode(frozenset({'zero', 'one'}), p=None, level=1), Remaining: ['two' 'three' 'four']
Traceback: 
[frozenset({'three', 'two'}), frozenset({'zero', 'one'})]
Grandchild 1
CombNode(frozenset({'four'}), p=frozenset({'three', 'two'}), level=3), Remaining: []
Grandchild Traceback: 
[frozenset({'four'}), frozenset({'three', 'two'}), frozenset({'zero', 'one'})]
GGchild: 
[]


In [39]:
# orig_list = [0,1,2,3,4]
orig_list = ["zero","one", "two", "three", "four"]
orig_list = np.array(orig_list)

binsizes = (2,2,1)
vidset = set(orig_list[ [0,1]] ) # len is equal to binsizes[0]

n0 = CombNode(vidset, None, orig_list, binsizes)
nprint("Root:", n0)

Root:
CombNode(frozenset({'zero', 'one'}), p=None, level=1), Remaining: ['two' 'three' 'four']


In [40]:
children = n0.get_children()
for i, c in enumerate(children):
    nprint(f"Child {i}", c)
    

Child 0
CombNode(frozenset({'three', 'two'}), p=frozenset({'zero', 'one'}), level=2), Remaining: ['four']
Child 1
CombNode(frozenset({'four', 'two'}), p=frozenset({'zero', 'one'}), level=2), Remaining: ['three']
Child 2
CombNode(frozenset({'four', 'three'}), p=frozenset({'zero', 'one'}), level=2), Remaining: ['two']


In [41]:
child = children[0]
nprint("Child Sample: ", child)

nprint("Check its parent node:")
print(child.parent)

nprint("Traceback: ",get_trace(child))

Child Sample: 
CombNode(frozenset({'three', 'two'}), p=frozenset({'zero', 'one'}), level=2), Remaining: ['four']
Check its parent node:
CombNode(frozenset({'zero', 'one'}), p=None, level=1), Remaining: ['two' 'three' 'four']
Traceback: 
[frozenset({'three', 'two'}), frozenset({'zero', 'one'})]


In [42]:
grandchildren = child.get_children()
grandchild = grandchildren[0]
nprint("Grandchild 1", grandchild)
nprint("Grandchild Traceback: ",get_trace(grandchild))

# Grandchildren has no child
ggchild = grandchild.get_children()
nprint("GGchild: ", ggchild)

Grandchild 1
CombNode(frozenset({'four'}), p=frozenset({'three', 'two'}), level=3), Remaining: []
Grandchild Traceback: 
[frozenset({'four'}), frozenset({'three', 'two'}), frozenset({'zero', 'one'})]
GGchild: 
[]


Awesome!

##  Finally, let's get all groupings
Apply `DFS` on `CombNode`s to get all possible groupings 

In [43]:
def get_all_combs(orig_list, binsizes, verbose=False):
    
    # First, get the level0 nodes and initiate the queue with them
    n = len(orig_list)
    search_depth = len(binsizes)
    level0_idxset_list = nCk(n, binsizes[0])
    level0_vidset_list = [ set(orig_list[list(idxset)])  for idxset in level0_idxset_list ]
    level0_nodes = [CombNode(vid=vidset, parent=None, orig_list=orig_list, binsizes=binsizes)
                   for vidset in level0_vidset_list]
    Q = level0_nodes
    
    if verbose: 
        print('orig_list: ', orig_list)
        print('binsizes: ', binsizes)
        nprint(f"Initial Queue of nodes [{len(level0_nodes)}]: \n", *Q)
        nprint()
    
    # Collect the possible combinations using DFS
    collection = set([])
    while len(Q) > 0:
        N = Q.pop(0)
        
        # this is better since it can handle binsizes that does not 
        # necessarily add upto the len(org_list) 
        if N.depth == search_depth: 
            collection.add(frozenset(get_trace(N)))
        else:
            children = N.get_children()
            
            ## debugging
            print(N)
            print('num children: ', len(children))
            print(children[0])
            temp = children[0]
            temp.get_children(verbose=True)
#             pdb.set_trace()
            ## end of debugging

            # add to the top of stack
            Q = children + Q
    if verbose:
        print("Num of all possible combs: ", len(collection)) 
        nprint("\nCollection: ", *collection, header=False)
    
    return collection 

In [44]:
def test_get_all_combs():
#     orig_list = np.array(['zero', 'one', 'two', 'three', 'four'])
    orig_list = np.array(range(5))

    #binsizes test 1
#     binsizes = [1,1,3]
#     result1 = get_all_combs(orig_list, binsizes, True)
    
#     binsizes test 2
    binsizes = [2,2,1]
    result2 = get_all_combs(orig_list, binsizes, True)
    
test_get_all_combs()




orig_list:  [0 1 2 3 4]
binsizes:  [2, 2, 1]
Initial Queue of nodes [10]: 

CombNode(frozenset({0, 1}), p=None, level=1), Remaining: [2 3 4]
CombNode(frozenset({0, 2}), p=None, level=1), Remaining: [1 3 4]
CombNode(frozenset({0, 3}), p=None, level=1), Remaining: [1 2 4]
CombNode(frozenset({0, 4}), p=None, level=1), Remaining: [1 2 3]
CombNode(frozenset({1, 2}), p=None, level=1), Remaining: [0 3 4]
CombNode(frozenset({1, 3}), p=None, level=1), Remaining: [0 2 4]
CombNode(frozenset({1, 4}), p=None, level=1), Remaining: [0 2 3]
CombNode(frozenset({2, 3}), p=None, level=1), Remaining: [0 1 4]
CombNode(frozenset({2, 4}), p=None, level=1), Remaining: [0 1 3]
CombNode(frozenset({3, 4}), p=None, level=1), Remaining: [0 1 2]
CombNode(frozenset({0, 1}), p=None, level=1), Remaining: [2 3 4]
num children:  3
CombNode(frozenset({2, 3}), p=frozenset({0, 1}), level=2), Remaining: [4]
remaining elements:  [4]
child binsize:  1
idxset_list:  [frozenset({0})]
vidset_list:  [{4}]
> <ipython-input-43-ee58

(Pdb)  q


BdbQuit: 

In [223]:
def test_get_all_combs_2():
    orig_list = np.array(range(7))
    binsizes = [1,1,5]
    result1 = get_all_combs(orig_list, binsizes, True)
    
test_get_all_combs_2()

orig_list:  [0 1 2 3 4 5 6]
binsizes:  [1, 1, 5]
Initial Queue of nodes [7]: 

CombNode(frozenset({0}), p=None, level=1), Remaining: [1 2 3 4 5 6]
CombNode(frozenset({1}), p=None, level=1), Remaining: [0 2 3 4 5 6]
CombNode(frozenset({2}), p=None, level=1), Remaining: [0 1 3 4 5 6]
CombNode(frozenset({3}), p=None, level=1), Remaining: [0 1 2 4 5 6]
CombNode(frozenset({4}), p=None, level=1), Remaining: [0 1 2 3 5 6]
CombNode(frozenset({5}), p=None, level=1), Remaining: [0 1 2 3 4 6]
CombNode(frozenset({6}), p=None, level=1), Remaining: [0 1 2 3 4 5]
CombNode(frozenset({0}), p=None, level=1), Remaining: [1 2 3 4 5 6]
num children:  6
CombNode(frozenset({1}), p=frozenset({0}), level=2), Remaining: [2 3 4 5 6]
> <ipython-input-222-7eee86332bb6>(36)get_all_combs()
-> Q = children + Q


(Pdb)  children[0].get_children()


[]


(Pdb)  temp = children[0]
(Pdb)  print(temp)


CombNode(frozenset({1}), p=frozenset({0}), level=2), Remaining: [2 3 4 5 6]


...

remaining elements:  [2 3 4 5 6]
child binsize:  5
idxset_list:  []
vidset_list:  []
[]


(Pdb)  nCk(5,5)


[]


(Pdb)  q


BdbQuit: 

In [219]:
def test_get_all_combs_3():
    orig_list = np.array(range(5))
    binsizes = [2,2]
    result1 = get_all_combs(orig_list, binsizes, True)

def test_get_all_combs_4():
    """
    Testcase:  binsizes does not sum up to len(orig_list)
    """
    orig_list = np.array(range(5))
    binsizes = [1,2]
    result1 = get_all_combs(orig_list, binsizes, True)
    
def test_get_all_combs_5():
    """
    Testcase:  binsizes does not sum up to len(orig_list)
    """
    orig_list = np.array(range(5))
    binsizes = [2,2]
    result1 = get_all_combs(orig_list, binsizes, True)

test_get_all_combs_3()

orig_list:  [0 1 2 3 4]
binsizes:  [2, 2]
Initial Queue of nodes [10]: 

CombNode(frozenset({0, 1}), p=None, level=1), Remaining: [2 3 4]
CombNode(frozenset({0, 2}), p=None, level=1), Remaining: [1 3 4]
CombNode(frozenset({0, 3}), p=None, level=1), Remaining: [1 2 4]
CombNode(frozenset({0, 4}), p=None, level=1), Remaining: [1 2 3]
CombNode(frozenset({1, 2}), p=None, level=1), Remaining: [0 3 4]
CombNode(frozenset({1, 3}), p=None, level=1), Remaining: [0 2 4]
CombNode(frozenset({1, 4}), p=None, level=1), Remaining: [0 2 3]
CombNode(frozenset({2, 3}), p=None, level=1), Remaining: [0 1 4]
CombNode(frozenset({2, 4}), p=None, level=1), Remaining: [0 1 3]
CombNode(frozenset({3, 4}), p=None, level=1), Remaining: [0 1 2]
Num of all possible combs:  15

Collection: 
frozenset({frozenset({3, 4}), frozenset({0, 2})})
frozenset({frozenset({2, 4}), frozenset({0, 3})})
frozenset({frozenset({1, 4}), frozenset({0, 3})})
frozenset({frozenset({0, 1}), frozenset({2, 3})})
frozenset({frozenset({0, 4}), fr

# SUCCESS!! >.<!!!

In [1]:
def solve_problem():
    orig_list = np.array(range(7))
#     orig_list = np.array( [ 'a','b','c','d', 'e','f','g'] )
    binsizes_list = [ [1,1,5], [1,2,4], [1,3,3], [2,2,3] ]
    all_combs = []
    for binsizes in binsizes_list:
        combs = get_all_combs(orig_list, binsizes)
        
        all_combs.extend(combs)
        nprint(f"binsizes: {binsizes}", len(combs))
    print("Total: ", len(all_combs))
    return all_combs
solve_problem()

NameError: name 'get_all_combs' is not defined

# RESUME FROM HERE!
todo:
Left at 1pm 
- [ ] debug the first two cases and check all the others are correct

---
## TODO

Timelog  
Left 3/24/2019 (Mon)  7:30pm    
Resumed 3/25/2019 (Tue) 10am  
Resumed on 3/26/2019 (W) 10am
Left 3/26/2019 (W) 1pm 

---
- [ ] debug the first two cases and check all the others are correct
---
- [ ] Turn this notebook into a blog post  (2 pomos)
---
- [x] actually nCk needs to run this for each vertex as the starting vid
- [x] This computation is needed at each horizontal level in the binning
---
- [ ] Incorporate this to the original binning problem (2 pomos)
    - [x] Define a new subclass of Node which keeps a set of integers as its vid
    - [ ] Find all possible groupsizes of size k:  
    For example, if I have a list of length 7, I need to know all possible ways to divide 7     into a list of k positive integers that sum up to 7: (1,1,5), (1,2,4), (1,3,3), (2,2,3)
    
---
Starting from 10pm?
- [x] Strang Linear Algebra
- [ ] Andrew Ng
- [ ] MM
- [ ] Murphy ch2,3
- [ ] Reinforcement Learning
- [ ] FastAI: UNet training